In [ ]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
# @assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
pkgs = [
    "Revise",
    # "FASTX",
    # # "VariantCallFormat",
    # "StatsBase",
    # "Distributions",
    # "StatsPlots",
    # "Random",
    # "Dates",
    # "DataFrames",
    # "BioSequences",
    # "Conda",
    # "Downloads"
]
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
# Pkg.develop(path="/global/cfs/projectdirs/m4269/cjprybol/Mycelia")
# Pkg.develop(path="../../..")
import Mycelia

In [ ]:
PROJECT_BASEDIR = dirname(pwd())
data_dir = joinpath(PROJECT_BASEDIR, "data")
genome_dir = mkpath(joinpath(data_dir, "genomes"))

In [ ]:
long_read_fastqs = sort(filter(x -> occursin(r"\.filtlong\.fq\.gz$", x), readdir(genome_dir, join=true)), by=x->filesize(x))

In [ ]:
threads = 8
for long_read_fastq in long_read_fastqs[end:end]
    outdir = mkpath(long_read_fastq * "_raven")
    outfile = joinpath(outdir, "assembly.fna")
    if !isfile(outfile) || (filesize(outfile) == 0)
        # --graphical-fragment-assembly
        @show basename(long_read_fastq)
        cmd =
        """
        $(Mycelia.MAMBA) run --live-stream -n raven-assembler raven --threads $(threads) $(long_read_fastq) > $(outfile)
        """
        Mycelia.nersc_sbatch(
            job_name = "$(basename(outdir))",
            mail_user = "cameron.prybol@gmail.com",
            logdir = mkpath("$(homedir())/workspace/slurmlogs"),
            qos = "shared",
            cpus_per_task= threads,
            cmd = cmd)
        # run(`$(Mycelia.MAMBA) run --live-stream -n raven-assembler raven --threads $(threads) $(long_read_fastq)`)
    else
        @info "$(outfile) already present..."
    end
end